# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [95]:
!pip3 install sympy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.7 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 80 kB/s eta 0:00:0101
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle
from numpy.linalg import lstsq
import sympy as sp

In [6]:
% matplotlib tk

UsageError: Line magic function `%` not found.


In [7]:
import matplotlib
matplotlib.use('TkAgg')

2024-03-29 16:52:38.761 Python[61435:16198670] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


# Provided functions

In [8]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.axis('off')
    plt.imshow(im)
    print(f'Set at least {min_lines} lines to compute vanishing point')
    print(f'The delete and backspace keys act like right clicking')
    print(f'The enter key acts like middle clicking')
    while True:
        print('Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print(f'Need at least {min_lines} lines, you have {n} now')
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [20]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g') # line (green)

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro') # vanishing point (red)
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [10]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # Use Singular Value Decomposition (SVD) to solve Ax = 0
    U, S, Vt = np.linalg.svd(lines.T)
    # The vanishing point is the right singular vector corresponding to the smallest singular value
    vanishing_point = Vt[-1]
    # Normalize to make the third coordinate 1
    vanishing_point /= vanishing_point[-1]

    return vanishing_point


In [11]:
def get_horizon_line(vpt1, vpt2):
    """
    Calculates the ground horizon line.
    """
    # line = np.cross(vpt1, vpt2)

    # a, b, _ = line
    # norm = np.sqrt(a**2 + b**2)
    # return line / norm

    # average y-coordinate
    y_avg = (vpt1[1] / vpt1[2] + vpt2[1] / vpt2[2]) / 2

    # 0x + 1y - y_avg = 0
    line = np.array([0, 1, -y_avg])

    return line

In [12]:
# def plot_horizon_line(ax, im, line):
#     """
#     Plots the horizon line.
#     """
#     height, width = im.shape[:2]
#     bx1, bx2 = 0, width
#     by1, by2 = 0, height

#     # Find intersections of the horizon line with the image borders
#     left = np.cross(np.array([1, 0, -bx1]), line)
#     right = np.cross(np.array([1, 0, -bx2]), line)
#     top = np.cross(np.array([0, 1, -by1]), line)
#     bottom = np.cross(np.array([0, 1, -by2]), line)

#     # Convert to inhomogeneous coordinates
#     pts = [left / left[2], right / right[2], top / top[2], bottom / bottom[2]]
#     # Filter points that are within the image boundary
#     pts = [pt for pt in pts if bx1 <= pt[0] <= bx2 and by1 <= pt[1] <= by2]

#     # Draw the horizon line
#     ax.imshow(im)
#     if len(pts) >= 2:
#         ax.plot([pts[0][0], pts[1][0]], [pts[0][1], pts[1][1]], 'r')

#     ax.set_xlim([bx1, bx2])
#     ax.set_ylim([by2, by1])

def plot_horizon_line(ax, im, line):
    """
    Plots the horizon line.
    """
    height, width = im.shape[:2]
    bx1, bx2 = 0, width

    # Since the line is horizontal, y value is constant and given by -c/b (line[2]/line[1])
    y_val = -line[2] / line[1]

    # Draw the horizon line
    ax.imshow(im)
    ax.plot([bx1, bx2], [y_val, y_val], 'r')  # Draw line across the image width at y_val height

    ax.set_xlim([bx1, bx2])
    ax.set_ylim([height, 0])  # Invert y-axis to match image coordinates

In [40]:
# def get_camera_parameters(vpts):
#     """
#     Computes the camera parameters. Hint: The SymPy package is suitable for this.
#     """
#     vpts = vpts / vpts[-1, :]
#     vpt1, vpt2, vpt3 = [sp.Matrix(vpt) for vpt in vpts.T]

#     px, py, k = sp.symbols('px py k')

#     A = sp.Matrix([[vpt1 + vpt2, vpt1 + vpt3, vpt2 + vpt3]])
#     b = sp.Matrix([vpt1.dot(vpt2), vpt1.dot(vpt3), vpt2.dot(vpt3)]) - sp.ones(3, 1)

#     solution = sp.solve(A.T * sp.Matrix([px, py, k]) - b, [px, py, k])

#     f = -2 * solution[k] - solution[px]**2 - solution[py]**2

#     K = sp.Matrix([[f, 0, solution[px]], [0, f, solution[py]], [0, 0, 1]])

#     return solution[k], solution[px], solution[py], np.array(K).astype(np.float32)

def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    vpts = vpts / vpts[-1, :]
    vpt1, vpt2, vpt3 = [sp.Matrix(vpt) for vpt in vpts.T]

    px, py, f = sp.symbols('px py f')
    x1, y1 = vpt1[0], vpt1[1]
    x2, y2 = vpt2[0], vpt2[1]
    x3, y3 = vpt3[0], vpt3[1]

    # Setup the equations based on the geometric constraints
    eq1 = sp.Eq((x1 - px) * (x2 - px) + (y1 - py) * (y2 - py) +f**2, 0)
    eq2 = sp.Eq((x1 - px) * (x3 - px) + (y1 - py) * (y3 - py) + f**2, 0)
    eq3 = sp.Eq((x3 - px) * (x2 - px) + (y3 - py) * (y2 - py) + f**2, 0)

    # Solve the system of equations
    solution = sp.solve((eq1, eq2, eq3), (px, py, f), dict=True)[1]

    K = sp.Matrix([[solution[f], 0, solution[px]], [0, solution[f], solution[py]], [0, 0, 1]])

    return solution[f], solution[px], solution[py], np.array(K).astype(np.float32)

In [14]:
def get_rotation_matrix(vpts, K):
    """
    Computes the rotation matrix using the camera parameters.
    """
    vpts = vpts / vpts[-1, :]
    K_inv = np.linalg.inv(K)
    R = np.zeros((3, 3))
    for i in range(len(vpts)):
        v_i = sp.Matrix(vpts[:, i])
        r_i = K_inv * v_i

        r_i = np.array(r_i).astype(np.float32)
        r_i = r_i.flatten()
        r_i = r_i / np.linalg.norm(r_i)

        R[:, i] = r_i
        print(f"r_{i}: {r_i}")
        
    return R

In [1]:
# def get_homography(...):
#     """
#     Compute homography for transforming the image into fronto-parallel 
#     views along the different axes.
#     """
#     # <YOUR CODE>
#     pass

# Main function

In [18]:
im = np.asarray(Image.open('./city.jpg'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('./data.pickle'):
    with open('./data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)
    num_vpts = 3

In [74]:
# # Click and save the line data for vanishing points. This snippet 
# # opens up an interface for selecting points and writes them to 
# # data.pickle file. The file is over-written.

# num_vpts = 3
# all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print(f'Getting vanishing point {i}')
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [41]:
# Part (1)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    vpts[:, i] = get_vanishing_point(all_lines[i])
    print(vpts[:, i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')

[-218.12925725  141.33875189    1.        ]
[914.30303401 119.05724645   1.        ]
[4.21082572e+02 3.53668775e+03 1.00000000e+00]


In [42]:
# Part (2) Computing and plotting the horizon
# method 1: draw a line that cross the two vanishing points
# method 2: averaging y coordinates of the two vanishing points, then draw a line with slope = 0
horizon_line = get_horizon_line(vpts[:, 0], vpts[:, 1])
print(f"horizon_line parameter: {horizon_line[0]:.4f}x + {horizon_line[1]:.4f}y + {horizon_line[2]:.4f} = 0")

fig = plt.figure(); ax = fig.gca()
plot_horizon_line(ax, im, horizon_line)
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

horizon_line parameter: 0.0000x + 1.0000y + -130.1980 = 0


In [43]:
# Part (3) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)

print(u, v, f, K)

355.906379457695 224.181564060017 558.418109511998 [[558.4181    0.      355.90637]
 [  0.      558.4181  224.18156]
 [  0.        0.        1.     ]]


In [44]:
# Part (4) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(vpts, K)
print(R)

r_0: [-0.71298563 -0.10289558  0.6935877 ]
r_1: [ 0.7009102  -0.13195407  0.70093715]
r_2: [0.01939838 0.9859009  0.16620192]
[[-0.71298563  0.70091021  0.01939838]
 [-0.10289558 -0.13195407  0.98590088]
 [ 0.69358772  0.70093715  0.16620192]]
